# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline , FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [2]:
# load data from database
engine = create_engine('sqlite:///disaster.db')
conn = engine.connect()
df = pd.read_sql_table('disaster_messages', conn)
#df = pd.DataFrame(engine.connect().execute(text('SELECT * FROM disaster_messages')))
X = df['message']
Y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    #Remove stop words to reduce training time
    #In fact this improves sensitivity of the model as i found out in the process
    clean_tokens = [w for w in clean_tokens if w not in stopwords.words("english")]
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, token_pattern=None)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
# train pipeline
pipeline.fit(X_train, Y_train)
# predict on test data
Y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
Y_pred_mod = pd.DataFrame(Y_pred, columns = Y.columns)

In [7]:
for col in Y_pred_mod.columns:
    print(col)
    print()
    print(classification_report(Y_test[col], Y_pred_mod[col], zero_division=0.0))

related

              precision    recall  f1-score   support

           0       0.73      0.32      0.44      1531
           1       0.82      0.96      0.88      4976
           2       1.00      0.09      0.16        47

    accuracy                           0.81      6554
   macro avg       0.85      0.46      0.50      6554
weighted avg       0.80      0.81      0.78      6554

request

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5419
           1       0.87      0.48      0.62      1135

    accuracy                           0.90      6554
   macro avg       0.88      0.73      0.78      6554
weighted avg       0.89      0.90      0.88      6554

offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6522
           1       0.00      0.00      0.00        32

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      

              precision    recall  f1-score   support

           0       0.98      0.99      0.98      5950
           1       0.88      0.77      0.82       604

    accuracy                           0.97      6554
   macro avg       0.93      0.88      0.90      6554
weighted avg       0.97      0.97      0.97      6554

cold

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6406
           1       0.78      0.05      0.09       148

    accuracy                           0.98      6554
   macro avg       0.88      0.52      0.54      6554
weighted avg       0.97      0.98      0.97      6554

other_weather

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6204
           1       0.64      0.05      0.09       350

    accuracy                           0.95      6554
   macro avg       0.79      0.52      0.53      6554
weighted avg       0.93      0.95      0.93      6554

### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect', CountVectorizer(token_pattern=None,
                   tokenizer=<function tokenize at 0x164405c60>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(token_pattern=None,
                 tokenizer=<function tokenize at 0x164405c60>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': None,
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth

In [11]:
# specify parameters for grid search
param_grid = {
    'clf__estimator__n_estimators': [100, 200, 300],
    'clf__estimator__max_depth': [None, 5, 10],
    'clf__estimator__min_samples_split': [2, 5, 10]
}

# Instantiate GridSearchCV
cv = GridSearchCV(pipeline, param_grid, cv=3)

I chose to only modify one parameter here because  it took a long time to calculate for more parameters. Also i believe to have realised a problem with GridSearchCV in combination with MultiOutputClassifier: GridSearchCV needs to evaluate the paramater combination by certain scorings. The optimal scoring to decide on is relatively difficult  to find because in MultiOutput you have different scores for each category. Also, i don't know yet which of the 36 scores exactly GridSearchCV takes to evaluate parameters.

In [12]:
cv.fit(X_train, Y_train)
Y_pred_cv = cv.predict(X_test)

KeyboardInterrupt: 

In [ ]:
#Try other parameters and RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV

parameters = {
    'tfidf__norm' : ['l1', 'l2'],
    'tfidf__smooth_idf': [True, False],
    'tfidf__sublinear_tf': [True, False],
    'tfidf__use_idf': [True, False],
    'clf__estimator__max_depth': [3, 5, 10],
    'clf__estimator__n_estimators': [50, 100, 500],
    'clf__estimator__verbose': [0, 5, 20]
}

cv1 = RandomizedSearchCV(pipeline, param_distributions=parameters, cv=3)

In [ ]:
cv1.fit(X_train, Y_train)
Y_pred_cv1 = cv1.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
Y_pred_cv_mod = pd.DataFrame(Y_pred_cv)
Y_pred_cv_mod.columns = Y_test.columns

for col in Y_pred_cv_mod.columns:
    print(col)
    print()
    print(classification_report(Y_test[col], Y_pred_cv_mod[col], zero_division=0.0))
    
print("\nBest Parameters:", cv.best_params_)

In [ ]:
Y_pred_cv1_mod = pd.DataFrame(Y_pred_cv1)
Y_pred_cv1_mod.columns = Y_test.columns

for col in Y_pred_cv1_mod.columns:
    print(col)
    print()
    print(classification_report(Y_test[col], Y_pred_cv1_mod[col], zero_division=0.0))
    
print("\nBest Parameters:", cv1.best_params_)

Baseline: GridSearchCV and RandomizedSearchCV did not lead to better results. In fact, sensitivity of the model was totally killed. Both searches lead to only zeros. It seems i did not choose the best combinations of Hyperparameters to dive into. Also maybe the scoring parameters lead the model to choosing not really the best parameter. For now it's too much for me to handle alone. If to dive deep into readings, which i will, to improve parameter settings like scoring and hyperparameter settings.

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

I want to try and include the clustering feature KMeans.

In [ ]:
from sklearn.cluster import KMeans
pipeline_new = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, token_pattern=None)),
    ('tfidf', TfidfTransformer()),
    ('kmeans', KMeans()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [ ]:
# train pipeline
pipeline_new.fit(X_train, Y_train)
# predict on test data
Y_pred = pipeline_new.predict(X_test)

In [ ]:
Y_pred_mod = pd.DataFrame(Y_pred, columns=Y_test.columns)

In [ ]:
for col in Y_pred_mod.columns:
    print(col)
    print()
    print(classification_report(Y_test[col], Y_pred_mod[col], zero_division=0.0))

The above results are somewhat worse then the original results. Maybe implementing the feature one after the other did weaken the effects of tfidf. Let's try a FeatureUnion of tfidf and KMeans.

In [ ]:
pipeline_new = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, token_pattern=None)),
    ('feature_union', FeatureUnion([
        ('tfidf', TfidfTransformer()),
        ('kmeans', KMeans()),
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [ ]:
# train pipeline
pipeline_new.fit(X_train, Y_train)
# predict on test data
Y_pred = pipeline_new.predict(X_test)

In [ ]:
Y_pred_mod = pd.DataFrame(Y_pred, columns=Y_test.columns)

In [ ]:
for col in Y_pred_mod.columns:
    print(col)
    print()
    print(classification_report(Y_test[col], Y_pred_mod[col], zero_division=0.0))

That's not a success. The precision and recall values are in fact a little lower than in the original results.
Let's export our original 'pipeline' then.

### 9. Export your model as a pickle file

In [ ]:
import pickle

pickle.dump(pipeline, open('disaster-response-model.pk1' , 'wb'))

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.